<div class="alert alert-success" role="alert">
  <h5>Our Libraries</h5>
</div>


In [ ]:
# pip  install yfinance

In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

from keras.models import Sequential
from tensorflow.keras import layers

# Optimizers
from tensorflow.keras.optimizers import Adam, RMSprop, Adadelta, SGD

%matplotlib inline
plt.rcParams.update({'font.size': 20})



<div class="alert alert-success" role="alert">
  <h5>Our Constants</h5>
</div>


In [ ]:
START = "2017-12-29"
END = "2022-12-15"
PAST_N_DAYS = 90
LEARNING_RATE = 0.001
EPOCHS = 20

<div class="alert alert-success" role="alert">
  <h5>Our Functions</h5>
</div>


In [ ]:
def Target_to_date_x_y(df):
    df_as_np = df.to_numpy()

    dates = df_as_np[:, 0]

    middle_matrix = df_as_np[:, 1:-1]
    X = middle_matrix.reshape((len(dates), middle_matrix.shape[1], 1))

    Y = df_as_np[:, -1]

    return dates, X.astype(np.float32), Y.astype(np.float32)

In [ ]:
def N_Target(df, first_date_str, last_date_str, n=3):
    
    first_date = pd.to_datetime(first_date_str)
    last_date  = pd.to_datetime(last_date_str)

    target_date = first_date

    dates = []
    X, Y = [], []

    last_time = False
    
    while True:
        df_subset = df.loc[:target_date].tail(n+1)

        if len(df_subset) != n+1:
            print(f'Error: Window of size {n} is too large for date {target_date}')
            return

        values = df_subset['Close'].to_numpy()
        x, y = values[:-1], values[-1]

        dates.append(target_date)
        X.append(x)
        Y.append(y)

        next_week = df.loc[target_date:target_date+datetime.timedelta(days=7)]
        next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
        next_date_str = next_datetime_str.split('T')[0]
        year_month_day = next_date_str.split('-')
        year, month, day = year_month_day
        next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))

        if last_time:
            break

        target_date = next_date

        if target_date == last_date:
            last_time = True
    
    ret_df = pd.DataFrame({})
    ret_df['Target Date'] = dates
  
    X = np.array(X)
    for i in range(0, n):
        X[:, i]
        ret_df[f'Target-{n-i}'] = X[:, i]
  
    ret_df['Target'] = Y

    return ret_df

In [ ]:
def check_stationarity(df):
    df = df.dropna()
    from statsmodels.tsa.stattools import adfuller
    result = adfuller(df, autolag='AIC')

    if result[0] < result[4].get("1%"):
        print("Passes on the 1% Critial Value")
        
    elif result[0] < result[4].get("5%"):
        print("Passes on the 5% Critial Value")
    
    elif result[0] < result[4].get("10%"):
        print("Passes on the 10% Critial Value")
    
    else:
        print("Failed")


In [ ]:
def do_stationarity(df):
    """ 
    This function takes a Non-stationarity DataFrame
    Returns a stationarity one by applying following equation
        df = df - df.shift(1)
    """
    df = df - df.shift(1)
    
    # Dropping the nulls, which is only the frist row in the dataframe
    df = df.dropna()
    
    return df

In [ ]:
def inv_stationarity(df_original, df_stationarity):
    df_original = df_original.dropna()

    """ 
    Inverse stationarity function takes a the original & Stationarity DataFrame
    Returns a Stationarity originally form
    """
    dfr = (df_stationarity.cumsum()*df_stationarity.iloc[-1])

    from sklearn import preprocessing

    scaler = preprocessing.MinMaxScaler(feature_range=(df_original.min()[0], df_original.max()[0]))

    names = dfr.columns
    d = scaler.fit_transform(dfr)
    dfr = pd.DataFrame(d, columns=names)
    return dfr

In [ ]:
def plotdf(df , mean = True):
    df = df.dropna()
    import matplotlib.pyplot as plt
    plt.rcParams["figure.figsize"] = (25,8)
    plt.title("CIB stock market price for Five years")
    plt.ylabel("Price in USD")
    plt.xlabel("Time in Years")
    plt.plot(df.index, df['Close']);
    if mean:
        plt.axhline(df['Close'].mean());

In [ ]:
def ADF_details(df):
    df = df.dropna()
    from statsmodels.tsa.stattools import adfuller
    result = adfuller(df, autolag='AIC')
    print(f'ADF Statistic: {result[0]}\n')
    print(f'p-value: {result[1]}\n')
    print('Critial Values:')
    for key, value in result[4].items():
        print(f'   {key}, {value}')

<div class="alert alert-success" role="alert">
  <h5>Pulling CIB stock market prices form yahoo finance API</h5>
</div>


In [ ]:
# Pulling the data form yahoo finance
#import yfinance as yf

import yfinance as yf
# Commercial International Bank in Egypt
df = yf.download('CIBEY', start = START, end = END)

[*********************100%***********************]  1 of 1 completed


<div class="alert alert-success" role="alert">
  <h5>Let's talk about the stock market</h5>
</div>


![https://datavizcatalogue.com/methods/images/anatomy/SVG/OHLC_chart.svg](attachment:OHLC_chart.svg)

In [ ]:
from plotly.offline import iplot
import plotly.graph_objs as go

trace = go.Ohlc(x=df.index,
                open=df.Open['1-2018'],
                high=df.High['1-2018'],
                low=df.Low['1-2018'],
                close=df.Close['1-2018'])

trace_all = go.Ohlc(x=df.index,
                open=df.Open,
                high=df.High,
                low=df.Low,
                close=df.Close)
# iplot([trace_all])
iplot([trace])

In [ ]:
df = df.filter(['Close'])

df = df.asfreq('b')

df = df.fillna(method="bfill")

In [ ]:
df.tail()

In [ ]:
plotdf(df)

In [ ]:
df.plot(kind='kde');

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm
plot_acf(df,lags=40)
plt.ylim(-0.3,1.3);

In [ ]:
plot_pacf(df,lags=10, method='ols')
plt.ylim(-0.3,1.3);

# MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler(feature_range=(0,1))

df = scaler.fit_transform(df)
df = pd.DataFrame(data = df, columns = ["Close"])

In [ ]:
time = pd.date_range(start='2017-12-29',end='2022-12-14',freq='B')
df_time = pd.DataFrame(data = time, columns = ["index"])
df.index = time

In [ ]:
Target_df = N_Target(df, 
                       '2018-06-01',
                       '2022-12-14',
                       n = PAST_N_DAYS)
Target_df

# REShaping

In [ ]:
dates, x, y = Target_to_date_x_y(Target_df)
y = y.reshape(-1, 1)
dates.shape, x.shape, y.shape

((1184,), (1184, 90, 1), (1184, 1))

In [ ]:
_80 = int(len(dates) * .8)
_90 = int(len(dates) * .9)

# Dates of the Train data & x_train & y_train => 80%
#(jan - 2018) upto (jan - 2022)
dates_train, x_train, y_train = dates[:_80], x[:_80], y[:_80]

#=============================================================

# Dates of the Validation & x_val & y_val => 10%
#(jan - 2022) upto (june - 2022)
dates_val, x_val, y_val = dates[_80:_90], x[_80:_90], y[_80:_90]


#=============================================================


# Dates of the Testing & x_Test & y_test => 10%
#(june - 2022) upto (dec - 2022)
dates_test, x_test, y_test = dates[_90:], x[_90:], y[_90:]


In [ ]:
plt.rcParams["figure.figsize"] = (25,8)
plt.title("Date Spilt \n")
plt.plot(dates_train, y_train)
plt.plot(dates_val, y_val)
plt.plot(dates_test, y_test)
plt.legend(['Train', 'Validation', 'Test']);

In [ ]:
df_train=pd.DataFrame(y_train, columns=['Close'])
df_train.index = dates_train

<div class="alert alert-success" role="alert">
  <h3 class="alert-heading">Let's Talk About STATIONARITY!</h3>
  <hr>
  <p class="mb-0"><p>The word "Stationarity" means</p>
    <br> &ensp; &ensp; &#9673  &ensp;  Our $\mu$ is constant.
    <br> &ensp; &ensp; &#9673  &ensp;  Our $\sigma$ is constant
    <br> &ensp; &ensp; &#9673  &ensp;  We have NO Seasonality
    </p>
</div>

In [ ]:
check_stationarity(df_train)

Failed


In [ ]:
ADF_details(df_train)

ADF Statistic: -2.160298590808073

p-value: 0.22096888332201436

Critial Values:
   1%, -3.4372887850912175
   5%, -2.8646033071530703
   10%, -2.568401081996585


<div class="alert alert-warning" role="alert">
    ADF Statistic < None of the critical values
    <br>                    
     Thus, We don't have Stationarity
</div>

In [ ]:
df_train_stationarity = do_stationarity(df_train)

In [ ]:
df_train_stationarity.describe()

In [ ]:
check_stationarity(df_train_stationarity)

Passes on the 1% Critial Value


In [ ]:
df_train_stationarity

In [ ]:
plotdf(df_train_stationarity)

In [ ]:
Target_df_train = N_Target(df_train_stationarity, 
                       '2018-01-18',
                       '2018-07-11',
                       n = PAST_N_DAYS)
Target_df_train

# dates_train, x_train, y_train = Target_to_date_x_y(Target_df_train)

Error: Window of size 90 is too large for date 2018-01-18 00:00:00


In [ ]:
x_train = x_train.reshape(947, 90,)
x_test = x_test.reshape(119, 90,)
x_val = x_val.reshape(118, 90,)
x_train.shape # RAWAN

(947, 90)

In [ ]:
y_train = y_train.reshape(947,) # RAWAN
y_test = y_test.reshape(119,) # RAWAN

# **SVR**

In [ ]:
from sklearn.svm import SVR
model = SVR(kernel = 'linear',gamma = 'scale', C=1.0, epsilon=0.1)
model.fit(x_train, y_train)

SVR(kernel='linear')

In [ ]:
train_predictions = model.predict(x_train)
train_predictions=scaler.inverse_transform(train_predictions.reshape(947, 1))
y_train=scaler.inverse_transform(y_train.reshape(947, 1))

plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.title("Training Error \n")
plt.ylabel("Price in USD\n")
plt.xlabel("\nTime in 6 Month period")
plt.rcParams["figure.figsize"] = (25,8)
plt.legend(['Training Predictions', 'Training Observations']);

In [ ]:
x_val.shape

In [ ]:
val_predictions = model.predict(x_val)

val_predictions=scaler.inverse_transform(val_predictions.reshape(118, 1))
y_val=scaler.inverse_transform(y_val.reshape(118, 1))

plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.rcParams["figure.figsize"] = (25,8)
plt.title("Validation Error \n")
plt.ylabel("Price in USD\n")
plt.xlabel("\nTime in Month")
plt.legend(['Validation Predictions', 'Validation Observations']);

In [ ]:
x_test.shape

In [ ]:
test_predictions = model.predict(x_test)

test_predictions=scaler.inverse_transform(test_predictions.reshape(119, 1))
y_test=scaler.inverse_transform(y_test.reshape(119, 1))


plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.rcParams["figure.figsize"] = (25,8)
plt.title("Testing Error \n")
plt.ylabel("Price in USD\n")
# plt.ylim(0.5,1.5)
plt.xlabel("\nTime in Month")

plt.legend(['Testing Predictions', 'Testing Observations']);

In [ ]:
# plt.plot(dates_train, train_predictions)
# plt.plot(dates_train, y_train)
plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.rcParams["figure.figsize"] = (25,8)
plt.title("Validation & Testing Error \n")
plt.ylabel("Price in USD\n")
plt.xlabel("\nTime in Month")
plt.legend(['Validation Predictions', 
            'Validation Observations',
            'Testing Predictions', 
            'Testing Observations']);